In [ ]:
import zipfile as zf
files = zf.ZipFile("/content/memes.zip", 'r')
files.extractall('meme')
files.close()

In [ ]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.6/286.6 kB 24.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pandas as pd
import easyocr
import cv2

# Initialize EasyOCR Reader for English, Hindi, and Marathi
reader = easyocr.Reader(['en', 'hi'], gpu=True)

# Initialize the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define the meme image directory
meme_directory = "/content/meme/memes"

# Function to classify toxicity based on image and text
def classify_toxicity(image_path, text):
    image = Image.open(image_path)
    inputs = processor(
        text=[text],
        images=image,
        return_tensors="pt",
        padding=True,
        truncation=True
    )
    outputs = model(**inputs)
    image_features = outputs.image_embeds
    text_features = outputs.text_embeds
    similarity = torch.cosine_similarity(image_features, text_features)
    toxicity_threshold = 0.2
    label = "Toxic" if similarity > toxicity_threshold else "Non-toxic"
    return label, similarity.item()

# Function to extract text from image using EasyOCR
def extract_text_from_image(image_path):
    result = reader.readtext(image_path, detail=1)
    extracted_text = " ".join([entry[1] for entry in result])
    return extracted_text.strip()

# Process all meme images in the directory
results = []
for filename in os.listdir(meme_directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(meme_directory, filename)
        text = extract_text_from_image(image_path)
        if not text:
            continue
        label, score = classify_toxicity(image_path, text)
        results.append({
            "image": filename,
            "extracted_text": text,
            "toxicity_label": label,
            "toxicity_score": score
        })

# Convert results into a DataFrame and save to CSV
df = pd.DataFrame(results)
df.to_csv("/content/meme/memes/meme_toxicity_results.csv", index=False)
print(df.head())


              image                                     extracted_text  \
0    520_NM_pic.jpg  IGdevilmemez_81७ cooamdep0 २s८l Ic०$ Monkeys क...   
1  151_M_pic_10.jpg  philmyyy We're both dead, but if we're born ag...   
2     43_NM_pic.jpg  When my Bestfriend doesn t tie a Friendship ba...   
3    514_NM_pic.jpg  *Someone :- Tum apne doston ko kis naam se bul...   
4   506_M_pic_2.jpg  १४ Y Ola Nibbi After Sex eleosturk Aaj Usne Mu...   

  toxicity_label  toxicity_score  
0          Toxic        0.372312  
1          Toxic        0.294317  
2          Toxic        0.311084  
3          Toxic        0.308909  
4          Toxic        0.297873  


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import os

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
CSV_FILE = "/content/meme/memes/meme_toxicity_results.csv"
IMAGE_DIR = "/content/meme/memes"


class MemeDataset(Dataset):
    def __init__(self, csv_file, image_dir, processor):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.processor = processor
        self.label_map = {"Non-toxic": 0, "Toxic": 1}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_dir, row["image"])
        text = row["extracted_text"]
        label = self.label_map[row["toxicity_label"]]

        # Load image
        image = Image.open(image_path).convert("RGB")

        return image, text, label

def collate_fn(batch):
    images, texts, labels = zip(*batch)
    inputs = processor(text=list(texts), images=list(images), return_tensors="pt", padding=True, truncation=True)
    labels = torch.tensor(labels, dtype=torch.long)  # Convert labels to tensor
    return inputs, labels

dataset = MemeDataset(CSV_FILE, IMAGE_DIR, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

class ToxicityClassifier(nn.Module):
    def __init__(self, clip_model):
        super(ToxicityClassifier, self).__init__()
        self.clip = clip_model
        self.fc = nn.Linear(512, 2)  # 512 is CLIP embedding size

    def forward(self, inputs):
        outputs = self.clip(**inputs)
        features = outputs.image_embeds + outputs.text_embeds  # Fuse image & text features
        logits = self.fc(features)
        return logits


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ToxicityClassifier(clip_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch_inputs, batch_labels in dataloader:
        batch_inputs = {key: val.to(device) for key, val in batch_inputs.items()}
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss / len(dataloader)}")

MODEL_PATH = "/content/meme/meme_toxicity_classifier.pth"
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved at: {MODEL_PATH}")


Epoch 1/5, Loss: 0.14073807795842488
Epoch 2/5, Loss: 0.07566362790763378
Epoch 3/5, Loss: 0.05720887634903193
Epoch 4/5, Loss: 0.0510748348881801
Epoch 5/5, Loss: 0.040846751412997644
✅ Model saved at: /content/meme/meme_toxicity_classifier.pth


In [ ]:

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)
model.eval()

def predict_toxicity(image_path, text, model, processor):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=[text], images=image, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        logits = model(inputs)
        prediction = torch.argmax(logits, dim=1).item()

    return "Toxic" if prediction == 1 else "Non-toxic"

test_image = "/content/19_M_pic_23.jpg"
# test_text = " meme text"
# result = predict_toxicity(test_image, test_text, model, processor)
# print(test_text)
# print("Prediction:", result)

test_text = extract_text_from_image(test_image)

# ✅ Now predict toxicity using real extracted text
result = predict_toxicity(test_image, test_text, model, processor)

print("Extracted Text:", test_text)
print("Prediction:", result)



<ipython-input-17-06ceeca24530>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=device))


Extracted Text: गधे की तो जिंदगी बन गई रे बाबा
Prediction: Toxic


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pandas as pd
import easyocr
import re


reader = easyocr.Reader(['en','hi'])


model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


meme_directory = "/content/meme/memes"


def preprocess_text(text):

    text = re.sub(r'\S+@\S+\.\S+', '', text)

    text = re.sub(r'http\S+|www\S+', '', text)

    text = re.sub(r'[^A-Za-z\s]', '', text)

    text = re.sub(r'\s+', ' ', text).strip()
    return text


def classify_toxicity(image_path, text):

    image = Image.open(image_path)


    inputs = processor(
        text=[text],
        images=image,
        return_tensors="pt",
        padding=True,
        truncation=True
    )


    outputs = model(**inputs)


    image_features = outputs.image_embeds
    text_features = outputs.text_embeds

    similarity = torch.cosine_similarity(image_features, text_features)
    toxicity_threshold = 0.4
    label = "Toxic" if similarity > toxicity_threshold else "Non-toxic"
    return label, similarity.item()

def extract_text_from_image(image_path):

    result = reader.readtext(image_path)

    text = " ".join([entry[1] for entry in result])

    text = preprocess_text(text)
    return text.strip()


results = []
for filename in os.listdir(meme_directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(meme_directory, filename)

        text = extract_text_from_image(image_path)
        if not text:
            continue

        label, score = classify_toxicity(image_path, text)

        results.append({
            "image": filename,
            "extracted_text": text,
            "toxicity_label": label,
            "toxicity_score": score
        })

df = pd.DataFrame(results)

df.to_csv("meme_toxicity_results_new1.csv", index=False)

print(df.head())

Progress: |██████████████████████████████████████████████████| 100.0% Complete              image                                     extracted_text  \
0    520_NM_pic.jpg        IGdevilmemez cooamdep sl Ic Monkeys Ooooaos   
1  151_M_pic_10.jpg  philmyyy Were both dead but if were born again...   
2     43_NM_pic.jpg  When my Bestfriend doesn t tie a Friendship ba...   
3    514_NM_pic.jpg  Someone Tum apne doston ko kis naam se bulate ...   
4   506_M_pic_2.jpg  Y Ola Nibbi After Sex eleosturk Aaj Usne Muje ...   

  toxicity_label  toxicity_score  
0          Toxic        0.301375  
1          Toxic        0.273048  
2          Toxic        0.325062  
3          Toxic        0.286841  
4          Toxic        0.291741  
